---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [7]:
import pandas as pd
import numpy as np

'''
PROCESS
    1. load files
    2. merge addresses with train and test. inner 
    3. remove rows with compliance = NaH from train
    4. determine which features to use. drop columns of 
        unwanted features for both train and test. reset indexes
        to 'ticket_id'
    5. make sure that remaining features all have values
        and not N/A or NaN. convert blanks in 'state' to 0
        all instances of blank for 'state' are because the owner
            address is out of the country
    6. since using scikit, need to convert string features 
        numbers. use pd.get_dummies for this
    7. need to make sure that train and text have the same columns
        iterate through train and compare with test. remove features
            from train that are not in test
        use a temporary set of train features to do this
    8. set X_train and y_train. 
        X_train is equal to train without the compliance row
        y_train is equal to the compliance row of train
    9. run a GradientBoostingClassifier
        then do predict_proba
        put result in a panda series
'''

def blight_model():
    
    # 1. LOAD AND CLEAN
    
    # load files
    
    train = pd.read_csv('train.csv', encoding='ISO-8859-1') 
    test = pd.read_csv('test.csv')
    address = pd.read_csv('addresses.csv')
    '''
    train = pd.read_csv('readonly/train.csv', encoding='ISO-8859-1',low_memory=False) 
    test = pd.read_csv('readonly/test.csv')
    address = pd.read_csv('readonly/addresses.csv')
    '''
    # merge addresses with train and test
    train = pd.merge(train, address, how='inner', left_on='ticket_id', right_on='ticket_id')
    test = pd.merge(test, address, how='inner', left_on='ticket_id', right_on='ticket_id') 
    
    # drop "compliance" NaN from train
    train = train[(train['compliance'] == 1) | (train['compliance'] == 0)]    
    
    # copy all train and text columns into a list. comment out the ones to keep
    cols_to_drop = [#'ticket_id',
                          'agency_name',
                          'inspector_name',
                          'violator_name',
                          'violation_street_number',
                          'violation_street_name',
                          'violation_zip_code','mailing_address_str_number',
                          'mailing_address_str_name',
                          'city', 
                          #'state',
                          'zip_code',
                          'non_us_str_code',
                          'country',
                          'ticket_issued_date',
                          'hearing_date',
                          #'violation_code', 
                          'violation_description',
                          #'disposition',
                          #'fine_amount',
                          #'admin_fee',
                          #'state_fee',
                          #'late_fee',
                          #'discount_amount',
                          'clean_up_cost',
                          #'judgment_amount',
                          'grafitti_status',
                          'address']
    
    # copy all test columns into a list. comment out the ones to keep
    train_cols_to_drop = ['payment_amount',
                         'payment_date',
                         'payment_status',
                         'balance_due',
                         'collection_status',
                         #'compliance',
                         'compliance_detail'] + cols_to_drop
    
    # drop unnessary columns. remaining ones are the features to test
    train = train.drop(train_cols_to_drop, axis=1).set_index('ticket_id')
    test = test.drop(cols_to_drop, axis=1).set_index('ticket_id')

    # clean up feature columns
    train = train.fillna(value={'state':0})
    #print(train.head())

    # need to convert text features into dummy variables
    train = pd.get_dummies(train, columns=['state', 'violation_code', 'disposition'])
    test = pd.get_dummies(test, columns=['state', 'violation_code', 'disposition'])
    
    # 2. CREATE X_train and y_train
    
    # need to make X_train and X_test the same shape
    # create a df of train features and a copy of it as a searchable set
    train_features = train.drop('compliance', axis=1)
    train_features_set = set(train_features)
   
    # iterate through through train_features and see if they are in test 
    # if not, then remove the feature from train_features_set
    for feature in set(train_features):
        if feature not in test:
            train_features_set.remove(feature)
    
    # convert to a list
    train_features_to_keep = list(train_features_set) 
    
    # set X_train, y_train, and X_test
    X_train = train[train_features_to_keep]
    y_train = train['compliance']
    X_test = test[train_features_to_keep]
    #print(X_train.shape, X_test.shape)
    
    
    # 3. run model
    
    from sklearn.ensemble import GradientBoostingClassifier
    clf = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
    y_proba = clf.predict_proba(X_test)[:,1]
    #result = 0
    result = pd.Series(y_proba, index=test.index)

    '''  
    # 4. determine auc
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(X_test, y_proba)
    roc_auc = auc(fpr, tpr)
    print(roc_auc)
    '''
    
    return result


In [8]:
blight_model()

C:\Users\wc5257\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3191: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


ticket_id
284932    0.142667
285362    0.017590
285361    0.051139
285338    0.043588
285346    0.062825
285345    0.043588
285347    0.053127
285342    0.883475
285530    0.014191
284989    0.021625
285344    0.050925
285343    0.017590
285340    0.017590
285341    0.050925
285349    0.062825
285348    0.043588
284991    0.021625
285532    0.020764
285406    0.020764
285001    0.027028
285006    0.017405
285405    0.018322
285337    0.016762
285496    0.050925
285497    0.043588
285378    0.018322
285589    0.021625
285585    0.043588
285501    0.051139
285581    0.017590
            ...   
376367    0.020546
376366    0.034705
376362    0.188561
376363    0.168848
376365    0.020546
376364    0.034705
376228    0.034705
376265    0.034705
376286    0.264453
376320    0.034705
376314    0.034705
376327    0.264453
376385    0.264453
376435    0.863699
376370    0.907636
376434    0.061868
376459    0.168854
376478    0.016270
376473    0.034705
376484    0.018768
376482    0.021399
37